### A Simple Data Pull From Twitter's REST API

In [9]:
import oauth2
from credentials import *
import json
import requests
import dataset

In [2]:
# assigning key variables
API_KEY = api_key
API_SECRET = api_secret
TOKEN_KEY = access_token
TOKEN_SECRET = access_token_secret

In [13]:
# function to create OAuth connection
def oauth_req(url, key, secret, http_method='GET', post_body="", http_headers=None):
    consumer = oauth2.Consumer(key=API_KEY, secret=API_SECRET)
    token = oauth2.Token(key=key, secret=secret)
    client = oauth2.Client(consumer, token)
    
    # Encode the post_body if it's a string
    if isinstance(post_body, str):
        post_body = post_body.encode('utf-8')
        
    resp, content = client.request(url, method=http_method, body=post_body, headers=http_headers)
    return content

url = 'https://api.twitter.com/1.1/search/tweets.json?q=%23childlabor'
data = oauth_req(url, TOKEN_KEY, TOKEN_SECRET)
print(data)

b'{"errors":[{"message":"You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product","code":453}]}\n'


In [14]:
print(type(data))

<class 'bytes'>


In [15]:
# to parse the data in this script
data_dict = json.loads(data)
print(type(data_dict))

<class 'dict'>


In [16]:
#or write the data to a file and parse it later
with open('tweet_data.json', 'wb') as data_file:
    data_file.write(data)

In [60]:
# we cannot use api v1.1, so we will switch to v2
BEARER_TOKEN = bearer_token

# Define the endpoint URL and parameters
search_url = "https://api.twitter.com/2/tweets/search/recent"
query_params = {'query': '#childlabor', 'tweet.fields': 'text,created_at'}

# Define headers including the Bearer token
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "User-Agent": "v2RecentSearchPython"
}

# Make the request to Twitter API v2
response = requests.get(search_url, headers=headers, params=query_params)

if response.status_code == 200:
    # Process the JSON response
    data = response.json()
    for tweet in data['data']:
        print(f"Tweet ID: {tweet['id']}")
        print(f"Text: {tweet['text']}")
        print(f"Created At: {tweet['created_at']}")
        print("------------------------")
else:
    print(f"Error: {response.status_code}, {response.text}")

Error: 403, {"client_id":"28062399","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API Access","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}


Unfortunatley this endpoint is not available in the free tier right now :(

## Advanced Data Collection from Twitter's REST API

In [4]:
import tweepy

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(TOKEN_KEY, TOKEN_SECRET)
api = tweepy.API(auth)
query = '#childlabor'
cursor = tweepy.Cursor(api.search_tweets, q=query, lang='en')

In [6]:
# iterate through the pages and save the data.
for page in cursor.pages():
    tweets = []
    for item in page:
        tweets.append(item._json)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

In [8]:
with open('hashchildlabor.json', 'wb') as outfile:
    json.dump(tweets, outfile)

NameError: name 'tweets' is not defined

In [10]:
def store_tweet(item):
    db = dataset.connect('sqlite:///data_wrangling.db')
    table = db['tweets']
    item_json = item._json.copy()
    for k, v in item_json.items():
        if isinstance(v, dict):
            item_json[k] = str(v)
    table.insert(item_json)

In [11]:
for page in cursor.pages():
    for item in page:
        store_tweet(item)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

## Advanced Data Collection from Twitter's Streaming API

In [ ]:
from tweepy.streaming import StreamingClient

class Listener(StreamingClient):
    def on_data(self, data):
        print(data)
        return True

In [62]:
streaming_client = Listener(BEARER_TOKEN)
streaming_client.add_rules(tweepy.StreamRule("child labor"))
streaming_client.filter()

Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

In [63]:
streaming_client.sample()

Stream encountered HTTP error: 403
HTTP error response text: {"client_id":"28062399","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API Access","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}
Stream encountered HTTP error: 403
HTTP error response text: {"client_id":"28062399","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API

KeyboardInterrupt: 